<a href="https://colab.research.google.com/github/rahat465/AttnGAN/blob/master/Text_to_image_generation_using_Bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install Bert liberray file for encoding the text

In [ ]:
!pip install simpletransformers

Download the dataset from kaggle

In [2]:
!pip install -q kaggle

In [3]:
import json 

In [ ]:
!pwd

/content


In [ ]:
%cd .

/content


In [4]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"rahatafza","key":"ae8d9125e6fcf829e542f4c03d1fddc7"}'}

In [5]:
!mkdir ~/.kaggle

In [ ]:
!rm ~/.kaggle

In [ ]:
!mv .kaggle /root/

mv: cannot stat '.kaggle': No such file or directory


In [7]:
!chmod 600 ~/.kaggle/kaggle.json

In [6]:
!cp kaggle.json ~/.kaggle

In [8]:
! kaggle datasets download -d aladdinpersson/flickr8kimagescaptions

 98% 1.01G/1.04G [00:07<00:00, 167MB/s]
100% 1.04G/1.04G [00:07<00:00, 152MB/s]


In [ ]:
!unzip /content/flickr8kimagescaptions.zip -d /content/T2I

In [10]:
%cd /content/T2I

/content/T2I


In [ ]:
!python train.py

Tryingto creat embeddings of the text files using bert

In [11]:
from transformers import BertTokenizer

# Load the BERT tokenizer.
print('Loading BERT tokenizer...')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

Loading BERT tokenizer...


Text tokens of the txt file enteries

In [12]:
from torch.utils.data import DataLoader, Dataset

In [13]:
import pandas as pd  # for lookup in annotation file

In [14]:
df = pd.read_csv("/content/T2I/flickr8k/captions.txt", delimiter='\t', header=None, names=['imagecaption'],skiprows=1)

In [ ]:
# Report the number of sentences.
print('Number of training sentences: {:,}\n'.format(df.shape[0]))

Number of training sentences: 40,455



In [ ]:
# Display 10 random rows from the data.
df.sample(10)

,imagecaption
11123,"254475194_3d8f4dfd53.jpg,A man is watching his..."
36710,"453756106_711c20471a.jpg,""A large , brown , fl..."
29559,"3490517179_76dbd690de.jpg,The small boy looks ..."
37211,"482047956_9a29e9cee6.jpg,Three girls are ridin..."
28582,"3453259666_9ecaa8bb4b.jpg,Two children swingin..."
31163,"3551447084_becc6a4666.jpg,A person riding a di..."
27124,"3397803103_8a46d716f4.jpg,Black dog catches ba..."
11703,"2586028627_ddd054d8cc.jpg,A small child wearin..."
36651,"448916362_17f3f1d0e1.jpg,A dog runs with his t..."
720,"115684808_cb01227802.jpg,Three men are setting..."


In [ ]:
df['caption']

0        1000268201_693b08cb0e.jpg,A child in a pink dr...
1        1000268201_693b08cb0e.jpg,A girl going into a ...
2        1000268201_693b08cb0e.jpg,A little girl climbi...
3        1000268201_693b08cb0e.jpg,A little girl climbi...
4        1000268201_693b08cb0e.jpg,A little girl in a p...
                               ...                        
40450    997722733_0cb5439472.jpg,A man in a pink shirt...
40451    997722733_0cb5439472.jpg,A man is rock climbin...
40452    997722733_0cb5439472.jpg,A person in a red shi...
40453    997722733_0cb5439472.jpg,A rock climber in a r...
40454    997722733_0cb5439472.jpg,A rock climber practi...
Name: caption, Length: 40455, dtype: object

In [ ]:
 
#dfSplitted[['image','caption']] = df['caption'].str.split(',', expand=True)

In [15]:
dfSplitted= df['imagecaption'].str.split(',', expand=True)

In [16]:
dfSplitted

,0,1,2,3,4,5,6
0,1000268201_693b08cb0e.jpg,A child in a pink dress is climbing up a set o...,None,None,None,None,None
1,1000268201_693b08cb0e.jpg,A girl going into a wooden building .,None,None,None,None,None
2,1000268201_693b08cb0e.jpg,A little girl climbing into a wooden playhouse .,None,None,None,None,None
3,1000268201_693b08cb0e.jpg,A little girl climbing the stairs to her playh...,None,None,None,None,None
4,1000268201_693b08cb0e.jpg,A little girl in a pink dress going into a woo...,None,None,None,None,None
...,...,...,...,...,...,...,...
40450,997722733_0cb5439472.jpg,A man in a pink shirt climbs a rock face,None,None,None,None,None
40451,997722733_0cb5439472.jpg,A man is rock climbing high in the air .,None,None,None,None,None
40452,997722733_0cb5439472.jpg,A person in a red shirt climbing up a rock fac...,None,None,None,None,None
40453,997722733_0cb5439472.jpg,A rock climber in a red shirt .,None,None,None,None,None


In [17]:
dfSplitted.columns = ['id','caption','c2','c3','c4','c5','c6']

Making a numpy array of captions

In [18]:
captions = dfSplitted.caption.values

Tokenization & Input Formatting

In [19]:
# Print the original sentence.
print(' Original: ', captions[1])

# Print the sentence split into tokens.
print('Tokenized: ', tokenizer.tokenize(captions[1]))

# Print the sentence mapped to token ids.
print('Token IDs: ', tokenizer.convert_tokens_to_ids(tokenizer.tokenize(captions[1])))

 Original:  A girl going into a wooden building .
Tokenized:  ['a', 'girl', 'going', 'into', 'a', 'wooden', 'building', '.']
Token IDs:  [1037, 2611, 2183, 2046, 1037, 4799, 2311, 1012]


Data Preparation for creating embeddings

The transformers library provides a helpful encode function which will handle most of the parsing and data prep steps for us.

Before we are ready to encode our text, though, we need to decide on a maximum sentence length for padding / truncating to.

The below cell will perform one tokenization pass of the dataset in order to measure the maximum sentence length.

In [20]:
max_len = 0

# For every sentence...
for sent in captions:

    # Tokenize the text and add `[CLS]` and `[SEP]` tokens.
    input_ids = tokenizer.encode(sent, add_special_tokens=True)

    # Update the maximum sentence length.
    max_len = max(max_len, len(input_ids))

print('Max sentence length: ', max_len)

Max sentence length:  39


Now we're ready to perform the real tokenization.

The tokenizer.encode_plus function combines multiple steps for us:

Split the sentence into tokens.
Add the special [CLS] and [SEP] tokens.
Map the tokens to their IDs.
Pad or truncate all sentences to the same length.
Create the attention masks which explicitly differentiate real tokens from [PAD] tokens.
The first four features are in tokenizer.encode, but I'm using tokenizer.encode_plus to get the fifth item (attention masks). Documentation is

In [21]:
import torch

In [23]:
# Tokenize all of the sentences and map the tokens to thier word IDs.
input_ids = []
attention_masks = []

# For every sentence...
for sent in captions:
    # `encode_plus` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    #   (5) Pad or truncate the sentence to `max_length`
    #   (6) Create attention masks for [PAD] tokens.
    encoded_dict = tokenizer.encode_plus(
                        sent,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = 64,           # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                   )
    
    # Add the encoded sentence to the list.    
    input_ids.append(encoded_dict['input_ids'])
    
    # And its attention mask (simply differentiates padding from non-padding).
    attention_masks.append(encoded_dict['attention_mask'])

# Convert the lists into tensors.
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
#labels = torch.tensor(labels)

# Print sentence 0, now as a list of IDs.
print('Original: ', captions[0])
print('Token IDs:', input_ids[0])

Original:  A child in a pink dress is climbing up a set of stairs in an entry way .
Token IDs: tensor([ 101, 1037, 2775, 1999, 1037, 5061, 4377, 2003, 8218, 2039, 1037, 2275,
        1997, 5108, 1999, 2019, 4443, 2126, 1012,  102,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0])


Embedding using Bert

IDs and encoding results are different values

In [26]:
from simpletransformers.language_representation import RepresentationModel
        
sentences = ["A child in a pink dress is climbing up a set of stairs in an entry way ."] #it should always be a list

model = RepresentationModel(
        model_type="bert",
        model_name="bert-base-uncased",
        use_cuda=True
    )

word_vectors = model.encode_sentences(sentences, combine_strategy=None)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForTextRepresentation: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForTextRepresentation from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTextRepresentation from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [28]:
word_vectors.shape

(1, 20, 768)

In [30]:
word_vectors[0].shape

(20, 768)